In [172]:
import pandas as pd
import numpy as np
import sklearn as sl
import re


from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [173]:
!pwd

/home/maxim/DS/Phase_0/Phase_0_final_proj/Phase0-Project/realises/Ver 3.0


In [174]:
data = pd.read_csv('/home/maxim/DS/Phase_0/Phase_0_final_proj/Phase0-Project/realises/Ver 2.0/Data')


In [175]:
data.drop(columns=['Unnamed: 0', 'Метро', 'building'], inplace=True)

In [176]:
data['walking_time_to_subway'].fillna(data['walking_time_to_subway'].median(), inplace=True)

/tmp/ipykernel_36999/654926195.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['walking_time_to_subway'].fillna(data['walking_time_to_subway'].median(), inplace=True)


In [177]:
def splitter(line):
    res=len(line.split(','))
    return res

In [178]:
data['additional_info'] = data['additional_info'].apply(splitter)

# Кодируем часть фич Target Encoder

In [179]:

class TargetEncoderRooms:
    def __init__(self, smoothing=1.0):
        self.smoothing = smoothing
        self.mapping = {}
        self.global_mean = None  # Добавляем атрибут для глобального среднего
        
    def fit_transform(self, X, y):
        self.global_mean = y.mean()  # Сохраняем глобальное среднее
        self.mapping = self._calculate_means(X, y)
        return self._apply_encoding(X)
        
    def transform(self, X):
        return self._apply_encoding(X)
        
    def _calculate_means(self, X, y):
        global_mean = y.mean()
        means = {}
        
        for room in X.unique():
            room_mean = y[X == room].mean()
            n = (X == room).sum()
            
            # Сглаживание для предотвращения переобучения
            means[room] = (room_mean * n + global_mean * self.smoothing) / (n + self.smoothing)
            
        return means
        
    def _apply_encoding(self, X):
        # Используем сохраненное глобальное среднее
        return X.map(self.mapping).fillna(self.global_mean)


# Создаем экземпляр кодировщика
encoder = TargetEncoderRooms(smoothing=1.0)




In [180]:
data['number_of_rooms'] = encoder.fit_transform(data['number_of_rooms'], data['price']) # Кодируем комнаты

# Кодируем часть фич LabelEncoder()

In [181]:
le = LabelEncoder()

In [182]:
data['type'] = le.fit_transform(data['type'])

In [183]:
data['balcony'] = le.fit_transform(data['balcony'])

In [184]:
data['bathroom'] = le.fit_transform(data['bathroom'])


In [185]:
data['subway_station_name'] = le.fit_transform(data['subway_station_name'])

In [186]:
data['lift'] = le.fit_transform(data['lift'])

# Кодируем часть фич OneHotEncoder

In [187]:
one = OneHotEncoder(sparse_output=False)  # drop='first' для dummy encoding


In [188]:

# Применяем кодирование
encoded_data = one.fit_transform(data[['windows']])
print("Закодированные данные:\n", encoded_data)

# Получаем названия новых столбцов
feature_names = one.get_feature_names_out(['windows'])
print("\nНазвания новых столбцов:", feature_names)

# Создаем новый DataFrame
temp = pd.DataFrame(encoded_data, columns=feature_names)

# Объединяем и удаляем исходный столбец
data = pd.concat([data, temp], axis=1)
data.drop(columns=['windows'], inplace=True)

Закодированные данные:
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]

Названия новых столбцов: ['windows_Во двор' 'windows_На улицу' 'windows_На улицу и двор']


In [189]:
# Применяем кодирование
encoded_data = one.fit_transform(data[['allow_children_pets']])

# Получаем названия новых столбцов
feature_names = one.get_feature_names_out(['allow_children_pets'])

# Создаем новый DataFrame
temp = pd.DataFrame(encoded_data, columns=feature_names)
data = pd.concat([data, temp], axis=1)

data.drop(columns=['allow_children_pets'], inplace=True)

In [190]:

# Применяем кодирование
encoded_data = one.fit_transform(data[['parking']])
print("Закодированные данные:\n", encoded_data)

# Получаем названия новых столбцов
feature_names = one.get_feature_names_out(['parking'])
print("\nНазвания новых столбцов:", feature_names)

# Создаем новый DataFrame
temp = pd.DataFrame(encoded_data, columns=feature_names)

# Объединяем и удаляем исходный столбец
data = pd.concat([data, temp], axis=1)
data.drop(columns=['parking'], inplace=True)

Закодированные данные:
 [[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]

Названия новых столбцов: ['parking_многоуровневая' 'parking_на крыше' 'parking_наземная'
 'parking_открытая' 'parking_подземная']


In [191]:

# Применяем кодирование
encoded_data = one.fit_transform(data[['renovation']])
print("Закодированные данные:\n", encoded_data)

# Получаем названия новых столбцов
feature_names = one.get_feature_names_out(['renovation'])
print("\nНазвания новых столбцов:", feature_names)

# Создаем новый DataFrame
temp = pd.DataFrame(encoded_data, columns=feature_names)

# Объединяем и удаляем исходный столбец
data = pd.concat([data, temp], axis=1)
data.drop(columns=['renovation'], inplace=True)

Закодированные данные:
 [[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

Названия новых столбцов: ['renovation_Без ремонта' 'renovation_Дизайнерский'
 'renovation_Евроремонт' 'renovation_Косметический']


In [192]:
data.rename(columns= {'windows_Во двор' : 'windows_1',
                      'windows_На улицу' : 'windows_2',
                      'windows_На улицу и двор': 'windows_3',
                      'allow_children_pets_Можно с детьми' : 'allow_children_pets_1',
                      'allow_children_pets_Можно с детьми, Можно с животными' : 'allow_children_pets_2',
                       'allow_children_pets_Можно с животными' : 'allow_children_pets_3',
                       'parking_многоуровневая' : 'parking_1',
                       'parking_на крыше' : 'parking_2',
                       'parking_наземная' : 'parking_3',
                       'parking_открытая' : 'parking_4',
                       'parking_подземная' : 'parking_5',
                       'renovation_Без ремонта' : 'renovation_1',
                       'renovation_Дизайнерский' : 'renovation_2',
                       'renovation_Евроремонт': 'renovation_3',
                       'renovation_Косметический' : 'renovation_4'

}, inplace=True)

In [ ]:
# data.to_csv('Batman & Robbin team')